In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
## Load the trined model, scaler pickle, onehot encoder pickle

model= load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [4]:
# example input data
input_data = {
    'CreditScore': 600,
    'Gender': 'Male',
    'Age': 42,
    'Tenure': 2,
    'Balance': 61921,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,
    'Geography': 'France'
}



In [7]:
# one hot encode 'Geography'

geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df 

c:\Users\YASH-SHIVGAN\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [9]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,600,Male,42,2,61921,1,1,1,50000,France


In [10]:
# Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,600,1,42,2,61921,1,1,1,50000,France


In [17]:
# Concatination one hot encoded
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,42,2,61921,1,1,1,50000,1.0,0.0,0.0


In [18]:
# scaling the input data 
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.29493847, -1.04241787, -0.22705408,
        -0.91668767,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [19]:
# predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step


array([[0.32796237]], dtype=float32)

In [20]:
prediction_proba =prediction[0][0]

In [21]:
prediction_proba

0.32796237

In [ ]:
if prediction_proba > 0.5 :
    print('The customer is likely to churn.')
else:
    print('The customer is unlikely to churn.')

The customer is unlikely to churn.


: 